# Python Tricks 
              Dan Bader 

In [ ]:
# price amount in cents to avoid currency rounding
shoes = {'name': 'Fancy Shoes', 'price': 14900} 

def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price']
    return price

apply_discount(shoes, 0.25), apply_discount(shoes, 0.5), apply_discount(shoes, 1.0), apply_discount(shoes, 1.5)

Assertions are meant to be *internal self-checks*. Pythons's assert statement is a debugging aid, not a mechanism for handling run-time errors.

In [ ]:
# Assertion implementation
if __debug__:
    if not expression1:
        raise AssertionError(expression2)


__debug__ is a global variable which is **true** under normal circumstances and **false** if optimizations are requested.

In computer programming jargon, a **heisenbug** is a software bug that seems to disappear or alter its behavior when one attempts to study it.

Assertions can be globally disabled with -O and -OO command lin switches as wll as `PYTHONOPTIMIZE` environment variable in CPython.

In [ ]:
def delete_product(prod_id, user):
    assert user.is_admin(),
    assert store.has_product(prod_id),
    store.get_product(prod_id).delete()

In [ ]:
# Better version
def delete_product(prod_id, user):
    if not user.is_admin():
        raise AuthError('Must be admin to delete')
    if not store.has_product(prod_id):
        raise ValueError('Unknow product id')
    store.get_product(prod_id).delete()

In [9]:
assert(1 == 2, 'This should fail')

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-9-2c057bd7fe24>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(1 == 2, 'This should fail')


In [10]:
assert 1 == 2, "this should fail"

AssertionError: this should fail

In [11]:
names = [
    'Alice',
    'Bob',
    'Dilbert'
    'Jane'
]
print(names)

['Alice', 'Bob', 'DilbertJane']


In Python, a comma can be placed after every item in a list, dict, or set constant, including the last item.

## Context Manager and with statement

`with` statement simplifies some common resource management patterns by abstracting their functionality and allowing them to be factored out and reused.

Another good example where the `with` statement is used effectively is `threading.Lock` class.

In [ ]:
with open('hello.txt', 'w') as f:
    f.write('Hello World!!')

A class can support `with` statement by implementing so-called _context managers_. Context manager is a simple protocol/interface that your object needs to follow in order to support `with` statement. 

A class/object must add `__enter__` and `__exit__` methods it it want to function as a context manager.


In [ ]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

In [ ]:
with ManagedFile('hello.txt') as f:
    f.write('hello world')
    f.write('good byte')

The `contextlib` utility module in the standard library provides a few more abstractions built on top of the basic context manager protocol.

In [12]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):  # generator()
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()
        
with managed_file('hello.txt') as f:
    f.write('hello and byte')

In [14]:
class Indenter:
    def __init__(self):
        self.level = 0
        self.spacing = [
            '....',
            '****',
            '++++',
        ]
    
    def __enter__(self):
        self.level += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
        
    def print(self, text):
        print('....' * self.level + text)
        

with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello!!')
        with indent:
            indent.print('bonjour')
    indent.print('bye')

....hi!
........hello!!
............bonjour
....bye


**Excercise:** implementing a context manager that measures the execution time of a code block using the time.time function. Be sure to try out writ ing both a decorator-based and a class-based variant to drive home the difference between the two

**Single Leading Underscore:** `_var` (Convention only)
Leading underscores impact how names get imported from modules. Python will _not_ import names with a leading underscore unless the module defines an `__all__` list

**Single Trailing Underscore:** `var_` 
Use by convention to avoid naming conflicts with Python keywords.

**Double Leading Underscore:** `__var` (Name Mangling)
Causes Python interpreter to rewrite the attribute names in order to avoid naming conflicts in subclasses. 

**Double Leading and Trailing Underscore:** `__var__` (No Name Mangling) 
Reserved for special use in the language.

**Single Underscore:** `_`
In REPLs, represents result of last expression evaluated by interpreter.

In [1]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 22

t = Test()
print(t.foo, t._bar)

11 22


In [ ]:
# my_module.py
def external_func():
    return 22

def _internal_func():
    return 42

from my_module import *   # Wild card imports should be avoided
external_func()   # OK
_internal_func()  # NameError

import my_module   # Regular import
my_module.external_func()
my_module._internal_func()  # OK

In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 22
        self.__baz = 44

class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'
        
t = Test()
print(dir(t))

t2 = ExtendedTest()
print(t2.foo, t2._bar, t2.__baz)

Most important skills for a programmer is "pattern recognition" and knowing where to look things up.

Double underscores are often referred to as **"dunders"** in Python community.

In [5]:
car = ('red', 'auto', 12, 3812.4)
color, _, _, mileage = car # _ as placeholder

Strings in Python have a unique built-in operation that can be accessed with the **% operator**.

In [11]:
errno = 50159747054
name = 'Bob'

# old style string formatting
print('Hey %s, there is a 0x%x error!' % (name, errno))
print('Hey %(name)s, there is a 0x%(errno)x error!' % {"name": name, "errno": errno })

# New style string formatting
print('Hey {}, there is a 0x{:x} error!'.format(name, errno))
print('Hey {name}, there is a 0x{errno:x} error!'.format(name=name, errno=errno))

# Literal string interpolation (Python 3.6+)


Hey Bob, there is a 0xbadc0ffee error!
Hey Bob, there is a 0xbadc0ffee error!
Hey Bob, there is a 0xbadc0ffee error!
Hey Bob, there is a 0xbadc0ffee error!


In [12]:
def greet(name, question):
    return f"Hello, {name}! How's it {question}?"

# gets translated into
# def greet(name, question):
#     return ("Hello, " + name + " ! How's it " + question + "?")

import dis
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 FORMAT_VALUE             0
              6 LOAD_CONST               2 ("! How's it ")
              8 LOAD_FAST                1 (question)
             10 FORMAT_VALUE             0
             12 LOAD_CONST               3 ('?')
             14 BUILD_STRING             5
             16 RETURN_VALUE


**grokking**: understand (something) intuitively or by empathy.
"corporate leaders seemed to grok this concept fairly quickly"

In [13]:
def yell(text):
    return text.upper() + '!'

bark = yell
del yell
print(bark('hi'))
print(bark.__name__)

HI!
yell


Python attaches a string identifier to every function at creation
time for debugging purposes. You can access this internal identifier with the `__name__` attribute.

Functions that can accept other functions as arguments are also called _higher-order functions_. They are a necessity for the functional programming style.

Python allows functions to be defined inside other functions.

The inner functions can also _capture and carry some of the parent function's state_ with them. Functions that captures parent's state are called **lexical closures** or just **closures**.

In [19]:
funcs = [bark, str.lower, str.capitalize]
for f in funcs:
    print(f, f('hey there'))
funcs[0]('hello!!')

def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)
    
greet(bark), greet(str.lower)

<function yell at 0x7fb2e85cbaf0> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there
HI, I AM A PYTHON PROGRAM!
hi, i am a python program


(None, None)

In [20]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

In [21]:
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)
print(speak('Hello, World'))

# whisper is not accessible outside speak
whisper('Yo')   # Error
speak.whisper   # Error

hello, world...


In [24]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper
    
f = get_speak_func(0.3)
print(f, f('hello'))

g = get_speak_func(0.7)
print(g, g('bye'))

<function get_speak_func.<locals>.whisper at 0x7fb2e85cb430> hello...
<function get_speak_func.<locals>.yell at 0x7fb2e85cbe50> BYE!


In [26]:
def get_speak_func(text, volume):
    def whisper():
        # inner function captures the 'text' argument of parent
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    
    if volume > 0.5:
        return yell
    else:
        return whisper
    
get_speak_func('Hello World', 0.8)()
    

'HELLO WORLD!'

In [27]:
def make_adder(n):
    def add(x):
        return x+n
    return add

plus_3 = make_adder(3)
plus_5 = make_adder(5)

print(plus_3(10))
print(plus_5(10))

13
15


class objects can be made _callable_, which allows objects to be treated like a functions. If an object is callable it means you can use the round parentheses function call syntax on it and even pass in function call arguments.

In [29]:
class Adder:
    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        return self.n + x
    
plus_3 = Adder(3)
print(plus_3(4)) # call object instance as function 

callable(plus_3)

7


True

The `lambda` keyword in Python provides a shortcut for declaring small anonymous functions. Lambda functions are restricted to a single expression. Lambda evaluates its expression and then automatically returns the expression's result.

In [30]:
(lambda x, y: x+y)(5, 4) # function expression

9

In [31]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
print(sorted(tuples))
print(sorted(tuples, key=lambda x: x[1]))

[(1, 'd'), (2, 'b'), (3, 'c'), (4, 'a')]
[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]


In [33]:
print(sorted(range(-5, 6)))
print(sorted(range(-5, 6), key=lambda x: x*x))

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]


Python’s **decorators** allow you to extend and modify the
behavior of a callable (functions, methods, and classes) without permanently modifying the callable itself.

Python's decorators _"decorate”_ or _“wrap”_ another function
and let you execute code _before_ and _after_ the wrapped function runs.

A decorator is a _callable that takes a callable as input and returns another callable_.

Putting an @null_decorator line in front of the function definition is the same as defining the function first and then running through the decorator.

Python’s `*args` and `**kwargs` feature for dealing with variable
numbers of arguments. `*` and `**` collect all positional and keyword arguments and stores them in variables `args` and `kwargs`. 

In [37]:
# Null decorator
def null_decorator(func):
    return func

def greet():
    return 'Hello'

greet = null_decorator(greet)

@null_decorator   # same as above explicit initialization
def greet():
    return 'Hello!'

print(greet())

# Another decorator
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

@uppercase
def greet():
    return 'Hello'

print(greet())

Hello!
HELLO


In [40]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

# Both are same 
# decorated_greet = strong(emphasis(greet))

@strong
@emphasis
def greet():
    return 'Hello!'

print(greet())

<strong><em>Hello!</em></strong>


In [44]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*argc, **kwargs)
    return wrapper


def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}()'
              f' with {args}, {kwargs}')
        
        original_result = func(*args, **kwargs)
        
        print(f'TRACE: {func.__name__}()' 
             f'returned {original_result}')
        return original_result
    return wrapper


@trace
def say(name, line):
    return f'{name}: {line}'

print(say('ABC', 'def'))

TRACE: calling say() with ('ABC', 'def'), {}
TRACE: say()returned ABC: def
ABC: def


One downside of decorator is that it "hides" some of the metadata attached to the original (undecorated) function.

Use `functools.wraps` decorator in user defined decorators to copy over the lost metadata from undecorated function.

In [52]:
def greet():
    """Return a friendly greeting"""
    return 'Hello'

decorated_greet = uppercase(greet)

# print(greet.__name__)
# print(greet.__doc__)
print(decorated_greet.__name__)
print(decorated_greet.__doc__)

# import functools

# def uppercase(func):
#     @functools.wraps(func)
#     def wrapper():
#         return func().upper()
#     return wrapper

# @uppercase
# def greet():
#     """Return a friendly greeting"""
#     return 'Hello'

# print(greet.__name__)
# print(greet.__doc__)
    

greet
Return a friendly greeting
